### Importing the Libraries used

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, StratifiedKFold
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier
import numpy as np
from sklearn.ensemble import VotingClassifier

### A bit of EDA

In [2]:
train= pd.read_csv("Train.csv")
test= pd.read_csv("test.csv")

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56000 entries, 0 to 55999
Data columns (total 52 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Applicant_ID    56000 non-null  object 
 1   form_field1     53471 non-null  float64
 2   form_field2     52156 non-null  float64
 3   form_field3     55645 non-null  float64
 4   form_field4     55645 non-null  float64
 5   form_field5     55645 non-null  float64
 6   form_field6     42640 non-null  float64
 7   form_field7     50837 non-null  float64
 8   form_field8     42640 non-null  float64
 9   form_field9     47992 non-null  float64
 10  form_field10    55645 non-null  float64
 11  form_field11    24579 non-null  float64
 12  form_field12    46105 non-null  float64
 13  form_field13    50111 non-null  float64
 14  form_field14    56000 non-null  int64  
 15  form_field15    33525 non-null  float64
 16  form_field16    42964 non-null  float64
 17  form_field17    44849 non-null 

In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24000 entries, 0 to 23999
Data columns (total 51 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Applicant_ID  24000 non-null  object 
 1   form_field1   22890 non-null  float64
 2   form_field2   22291 non-null  float64
 3   form_field3   23854 non-null  float64
 4   form_field4   23854 non-null  float64
 5   form_field5   23854 non-null  float64
 6   form_field6   18396 non-null  float64
 7   form_field7   21769 non-null  float64
 8   form_field8   18396 non-null  float64
 9   form_field9   20600 non-null  float64
 10  form_field10  23853 non-null  float64
 11  form_field11  10602 non-null  float64
 12  form_field12  19817 non-null  float64
 13  form_field13  21537 non-null  float64
 14  form_field14  24000 non-null  int64  
 15  form_field15  14408 non-null  float64
 16  form_field16  18526 non-null  float64
 17  form_field17  19305 non-null  float64
 18  form_field18  19631 non-nu

In [5]:
train.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.000,0.0,10689720.0,252072.0,4272776.0,...,0.392854,2.02,0.711632,0.0,0.0,charge,NaN,1.129518,0.044335,no
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.000,0.0,898979.0,497531.0,9073814.0,...,0.314281,8.08,0.183584,NaN,0.0,charge,349.805730,1.620483,0.322436,no
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0,6.282,NaN,956940.0,NaN,192944.0,...,0.162965,18.18,0.791136,0.0,0.0,charge,NaN,1.513370,0.011640,yes
3,Apcnt_1000012,3372.0,0.17005,0.5050,0.0,0.000,192166.0,3044703.0,385499.0,3986472.0,...,0.488884,2.02,0.685168,NaN,0.0,charge,89.940100,0.664452,0.082729,no
4,Apcnt_1000016,3370.0,0.77270,1.1010,0.0,0.000,1556.0,214728.0,214728.0,1284089.0,...,0.275000,12.12,0.438168,0.0,0.0,charge,97.887502,1.427891,0.045630,no


In [6]:
test.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50
0,Apcnt_1000032,3236.0,0.34875,10.2006,0.0000,0.0,418564.0,418564.0,418564.0,540710.0,...,NaN,0.825000,1.01,0.800000,NaN,0.0,charge,NaN,0.000000,0.011221
1,Apcnt_1000048,3284.0,1.27360,2.9606,9.0198,0.0,0.0,9858816.0,49014.0,1510098.0,...,18.8415,0.507694,4.04,0.623248,1.0,0.0,lending,NaN,0.504974,0.043525
2,Apcnt_1000052,NaN,0.27505,0.0600,0.0000,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,0.00,NaN,NaN,NaN,charge,NaN,0.000000,NaN
3,Apcnt_1000076,3232.0,0.28505,2.8032,0.0000,0.0,0.0,473802.0,473802.0,1724437.0,...,NaN,0.916663,2.02,0.464224,NaN,NaN,charge,90.163742,0.788809,0.104029
4,Apcnt_1000080,3466.0,2.09545,0.8318,2.5182,0.0,19839.0,1150662.0,1150662.0,7860523.0,...,NaN,0.234047,23.23,0.726688,0.0,0.0,lending,1303.587148,1.637733,0.163124


### Encoding the categorical objects to be used

In [7]:
train["form_field47"] = LabelEncoder().fit_transform(train["form_field47"])
train["default_status"] = LabelEncoder().fit_transform(train["default_status"])
test["form_field47"] = LabelEncoder().fit_transform(test["form_field47"])

In [8]:
X= train.drop(["Applicant_ID", "default_status"], axis=1)
y= train["default_status"]
x_test= test.drop("Applicant_ID", axis=1) 

### A bit of Feature Engineering 

In [9]:
Feat= X.copy()
Feat["1st"]=X["form_field21"]-(X.form_field22+X.form_field23)
Feat["2nd"]= X.form_field8-X.form_field6
for i in range(len(x_test.form_field1)):
    if X.form_field28[i]<300 and X.form_field47[i]==0:
        Feat["3rd"]=1
    else:
        Feat["3rd"]=0
Feat["4th"]= X.form_field38- X.form_field36

In [10]:
Feat_test= x_test.copy()
Feat_test["1st"]=x_test["form_field21"]-(x_test.form_field22+x_test.form_field23)
Feat_test["2nd"]= x_test.form_field8-x_test.form_field6
for i in range(len(x_test.form_field1)):
    if x_test.form_field28[i]<300 and x_test.form_field47[i]==0:
        Feat_test["3rd"]=1
    else:
        Feat_test["3rd"]=0
Feat_test["4th"]= x_test.form_field38- x_test.form_field36

In [11]:
Feat.drop(["form_field48", "form_field49"], inplace=True, axis=1)
Feat_test.drop(["form_field48", "form_field49"], inplace=True, axis=1)

### Training and testing the models with hyperparameters

Running Hyperparameter testing with GridsearchCV was too computationally expensive and time consuming as I was using CPU, so I was limited in tuning my parameters.

###  Voting Classifier with 2 Catboost models and 1 LGBM  

In [12]:
Catbstfine= CatBoostClassifier(silent=True, eval_metric="AUC", iterations= 2000, learning_rate=0.01, l2_leaf_reg=3.5,
                            depth=8, rsm=0.98, random_seed=2019)
catbst2000= CatBoostClassifier(silent=True, eval_metric="AUC", iterations= 2000, random_seed=2019)
LGB= LGBMClassifier(silent=True, eval_metric="AUC", random_seed=2019)

In [13]:
clf= VotingClassifier(
    estimators= [
        ("catbstfine", Catbstfine),
        ("LGBM", LGB),
        ("catbst2000", catbst2000)
    ],
    voting="soft"
)

In [14]:
test_pred=[]
y_preds=[]
err=[]
splits= None
fold= StratifiedKFold(n_splits=10, shuffle= True, random_state=2019)


for train_index, test_index in fold.split(Feat,y):
    xtrain, xval = Feat.iloc[train_index], Feat.iloc[test_index]
    ytrain, yval= y.iloc[train_index], y.iloc[test_index]
    
    clf.fit(xtrain, ytrain)
    predic= clf.predict_proba(xval)[:,1]
    test_pred.append(predic)
    preds= clf.predict_proba(Feat_test)[:,1]
    y_preds.append(preds)

[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: eval_metric


In [15]:
y_preds = np.mean(y_preds, axis=0)
VotingClassifierwithFeat= pd.DataFrame({"Applicant_ID":test["Applicant_ID"], "default_status": y_preds})
VotingClassifierwithFeat.head()

,Applicant_ID,default_status
0,Apcnt_1000032,0.314036
1,Apcnt_1000048,0.354162
2,Apcnt_1000052,0.396775
3,Apcnt_1000076,0.746862
4,Apcnt_1000080,0.140766


In [16]:
VotingClassifierwithFeat.to_csv("vcf_feat.csv", index= False)

In [20]:
pd.read_csv("CatboostStrK.csv")

,Applicant_ID,default_status
0,Apcnt_1000032,0.292344
1,Apcnt_1000048,0.375840
2,Apcnt_1000052,0.390553
3,Apcnt_1000076,0.732073
4,Apcnt_1000080,0.186724
...,...,...
23995,Apcnt_999940,0.684821
23996,Apcnt_999956,0.268720
23997,Apcnt_999976,0.279096
23998,Apcnt_999984,0.500526
